# Load dataset

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
from transformers import AutoTokenizer
from transformers import BertForSequenceClassification
from transformers import BertTokenizer
from sentence_transformers import SentenceTransformer

import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
vietnamese_history_dataset_path = "../../datasets/vietnamese_history_dataset/data.json"
df = pd.read_json(vietnamese_history_dataset_path)
df.head()

,title,content,type
0,BUỔI ĐẦU LỊCH SỬ NƯỚC TA,Chương: BUỔI ĐẦU LỊCH SỬ NƯỚC TA,chapter
1,THỜI NGUYÊN THUỶ TRÊN ĐẤT NƯỚC TA,Chương: BUỔI ĐẦU LỊCH SỬ NƯỚC TA\nBài: THỜI NG...,lesson
2,Những dấu tích của Người tối cổ được tìm thấy ...,Bài: THỜI NGUYÊN THUỶ TRÊN ĐẤT NƯỚC TA\nNhững ...,title
3,"Ở giai đoạn đầu, Người tinh khôn sống như thế ...",Bài: THỜI NGUYÊN THUỶ TRÊN ĐẤT NƯỚC TA\nỞ giai...,title
4,Giai đoạn phát triển của Người tinh khôn có gì...,Bài: THỜI NGUYÊN THUỶ TRÊN ĐẤT NƯỚC TA\nGiai đ...,title


In [16]:
df["content_embedding"] = None
df["embeded"] = None

for idx in range(len(df)):
    df["content_embedding"] = df["title"][idx] + " " + df["content"][idx]

df.head()

,title,content,type,content_embedding,embeded
0,BUỔI ĐẦU LỊCH SỬ NƯỚC TA,Chương: BUỔI ĐẦU LỊCH SỬ NƯỚC TA,chapter,Thực hiện kế hoạch 5 năm 1996 - 2000 Bài: Đất ...,None
1,THỜI NGUYÊN THUỶ TRÊN ĐẤT NƯỚC TA,Chương: BUỔI ĐẦU LỊCH SỬ NƯỚC TA\nBài: THỜI NG...,lesson,Thực hiện kế hoạch 5 năm 1996 - 2000 Bài: Đất ...,None
2,Những dấu tích của Người tối cổ được tìm thấy ...,Bài: THỜI NGUYÊN THUỶ TRÊN ĐẤT NƯỚC TA\nNhững ...,title,Thực hiện kế hoạch 5 năm 1996 - 2000 Bài: Đất ...,None
3,"Ở giai đoạn đầu, Người tinh khôn sống như thế ...",Bài: THỜI NGUYÊN THUỶ TRÊN ĐẤT NƯỚC TA\nỞ giai...,title,Thực hiện kế hoạch 5 năm 1996 - 2000 Bài: Đất ...,None
4,Giai đoạn phát triển của Người tinh khôn có gì...,Bài: THỜI NGUYÊN THUỶ TRÊN ĐẤT NƯỚC TA\nGiai đ...,title,Thực hiện kế hoạch 5 năm 1996 - 2000 Bài: Đất ...,None


# Load tokenizer model

In [ ]:
tokenizer = BertTokenizer.from_pretrained(
    "trituenhantaoio/bert-base-vietnamese-uncased",
    cache_dir="../../tokenizer_models/"
)

# Chungking data về kích thước 512 tokens

In [ ]:
def chunk_text(text, max_length=512, max_tokens=512, stride=64, tokenizer=None):
    """Split text into chunks of max_length tokens."""
    if text is None:
        raise ValueError("Text must be provided.")
    if max_length > 512:
        raise ValueError("max_length must be less than or equal to 512.")
    if max_tokens > 512:
        raise ValueError("max_tokens must be less than or equal to 512.")
    if tokenizer is None:
        raise ValueError("Tokenizer must be provided.")
    
    tokens = tokenizer.encode(text, add_special_tokens=True)
    chunks = []
    start = 0
    while start < len(tokens):
        end = start + max_tokens
        chunk = tokens[start:end]
        chunks.append(chunk)

        # Nếu có stride (chồng lấn token giữa các đoạn)
        start += max_tokens - stride if stride > 0 else max_tokens

    # Decode lại các đoạn thành văn bản
    chunk_texts = [tokenizer.decode(chunk, skip_special_tokens=True) for chunk in chunks]
    return chunk_texts

In [ ]:
text = "Đây là một đoạn văn bản tiếng Việt rất dài..." * 500
chunks = chunk_text(text, max_tokens=512, stride=100, tokenizer=tokenizer)

for i, chunk in enumerate(chunks):
    print(f"--- Chunk {i+1} ---")
    print(chunk[:200], "...")  # In thử 200 ký tự đầu


In [25]:
# Create a new dataframe
df_chunks = pd.DataFrame(columns=["title", "content", "content_embedding", "chunk_embedding", "embeded"])

for idx in range(len(df)):
    chunks = chunk_text(df["content_embedding"][idx], max_tokens=512, stride=64, tokenizer=tokenizer)
    for chunk in chunks:
        len_df_chunks = len(df_chunks)
        df_chunks.loc[len_df_chunks, "title"] = df["title"][idx]
        df_chunks.loc[len_df_chunks, "content"] = df["content"][idx]
        df_chunks.loc[len_df_chunks, "content_embedding"] = df["content_embedding"][idx]
        df_chunks.loc[len_df_chunks, "chunk_embedding"] = chunk
        df_chunks.loc[len_df_chunks, "embeded"] = None
    
df_chunks.head()

,title,content,content_embedding,chunk_embedding,embeded
0,BUỔI ĐẦU LỊCH SỬ NƯỚC TA,Chương: BUỔI ĐẦU LỊCH SỬ NƯỚC TA,Thực hiện kế hoạch 5 năm 1996 - 2000 Bài: Đất ...,thuc hien ke hoach 5 nam 1996 2000 bai đat nuo...,None
1,BUỔI ĐẦU LỊCH SỬ NƯỚC TA,Chương: BUỔI ĐẦU LỊCH SỬ NƯỚC TA,Thực hiện kế hoạch 5 năm 1996 - 2000 Bài: Đất ...,nganh kinh te tung buoc chuyen dich theo huong...,None
2,BUỔI ĐẦU LỊCH SỬ NƯỚC TA,Chương: BUỔI ĐẦU LỊCH SỬ NƯỚC TA,Thực hiện kế hoạch 5 năm 1996 - 2000 Bài: Đất ...,chat luong san pham chua tot gia thanh cao hie...,None
3,BUỔI ĐẦU LỊCH SỬ NƯỚC TA,Chương: BUỔI ĐẦU LỊCH SỬ NƯỚC TA,Thực hiện kế hoạch 5 năm 1996 - 2000 Bài: Đất ...,,None
4,THỜI NGUYÊN THUỶ TRÊN ĐẤT NƯỚC TA,Chương: BUỔI ĐẦU LỊCH SỬ NƯỚC TA\nBài: THỜI NG...,Thực hiện kế hoạch 5 năm 1996 - 2000 Bài: Đất ...,thuc hien ke hoach 5 nam 1996 2000 bai đat nuo...,None


# Embedding data sử dụng model all-MiniLM-L6-v2

In [29]:
import glob

glob.glob("../../embedding_models/*")

['../../embedding_models\\all-MiniLM-L6-v2',
 '../../embedding_models\\distiluse-base-multilingual-cased-v2',
 '../../embedding_models\\gte-multilingual-base',
 '../../embedding_models\\multilingual-e5-small',
 '../../embedding_models\\README.md',
 '../../embedding_models\\sup-SimCSE-VietNamese-phobert-base',
 '../../embedding_models\\vietnamese-bi-encoder',
 '../../embedding_models\\vietnamese-embedding']

In [ ]:
from transformers import AutoTokenizer
from transformers import AutoModel


tokenizer = AutoTokenizer.from_pretrained(
    "all-MiniLM-L6-v2",
    cache_dir="../../embedding_models/"
)

model = AutoModel.from_pretrained(
    "all-MiniLM-L6-v2",
    cache_dir="../../embedding_models/"
)

In [28]:
for idx in range(len(df_chunks)):
    df_chunks["embeded"][idx] = tokenizer.encode(
        df_chunks["chunk_embedding"][idx],
        return_tensors='pt',
        truncation=True,
        max_length=512
    )

df_chunks.head()

,title,content,content_embedding,chunk_embedding,embeded
0,BUỔI ĐẦU LỊCH SỬ NƯỚC TA,Chương: BUỔI ĐẦU LỊCH SỬ NƯỚC TA,Thực hiện kế hoạch 5 năm 1996 - 2000 Bài: Đất ...,thuc hien ke hoach 5 nam 1996 2000 bai đat nuo...,"[[tensor(2), tensor(146), tensor(659), tensor(..."
1,BUỔI ĐẦU LỊCH SỬ NƯỚC TA,Chương: BUỔI ĐẦU LỊCH SỬ NƯỚC TA,Thực hiện kế hoạch 5 năm 1996 - 2000 Bài: Đất ...,nganh kinh te tung buoc chuyen dich theo huong...,"[[tensor(2), tensor(7439), tensor(416), tensor..."
2,BUỔI ĐẦU LỊCH SỬ NƯỚC TA,Chương: BUỔI ĐẦU LỊCH SỬ NƯỚC TA,Thực hiện kế hoạch 5 năm 1996 - 2000 Bài: Đất ...,chat luong san pham chua tot gia thanh cao hie...,"[[tensor(2), tensor(4866), tensor(17430), tens..."
3,BUỔI ĐẦU LỊCH SỬ NƯỚC TA,Chương: BUỔI ĐẦU LỊCH SỬ NƯỚC TA,Thực hiện kế hoạch 5 năm 1996 - 2000 Bài: Đất ...,,"[[tensor(2), tensor(3)]]"
4,THỜI NGUYÊN THUỶ TRÊN ĐẤT NƯỚC TA,Chương: BUỔI ĐẦU LỊCH SỬ NƯỚC TA\nBài: THỜI NG...,Thực hiện kế hoạch 5 năm 1996 - 2000 Bài: Đất ...,thuc hien ke hoach 5 nam 1996 2000 bai đat nuo...,"[[tensor(2), tensor(146), tensor(659), tensor(..."


In [44]:
from transformers import BertModel
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained(
    "trituenhantaoio/bert-base-vietnamese-uncased",
    cache_dir="../../embedding_models",
)
model = BertModel.from_pretrained(
    "trituenhantaoio/bert-base-vietnamese-uncased",
    cache_dir="../../embedding_models",
)

In [45]:
sentence = df_chunks["chunk_embedding"][10]
inputs = tokenizer(
    sentence,
    return_tensors="pt",
    truncation=True,
    max_length=512,
    padding="max_length",
)

In [46]:
print(inputs)

{'input_ids': tensor([[    2,  4866, 17430,  1642,  8335,  2083,  1134,   138,    60,   152,
           110, 20716,    99,  1555,  1460,   659,  1499,   449,  1936,   521,
           158,  5316,  1197,  3086,  3332,  2083,  2975,  3332,   616,  1721,
         25199,  1105,  1964,   687,  3120,   879,  3317,   158,  5316,   389,
           521,  1039,  2083,  2291,   396,   659,   358,   138,  4308,   497,
          9015,  1555,  2668,   819,  2083,   639,   521,  1254,  1134,   138,
          6135,   237,  3488,  2507,  1460, 26069,  2668, 26069,   358, 17479,
          1414,   358,  3059,   585,  1555,   448,  3351,  1134,  9237,  2606,
          2528,  6689, 26069,  1861,   152,   170,  1555,   170,  8740,   315,
          6353,  1674,  1861,  5701,  3466,    97,  1861,  1737,   659,   110,
          1737,   659,   844,  2507,  1851,  4169,  1197,   138,   920,  5701,
          4169,  1861, 13553,   482,  3628,    97,  1936,   521,    10,  3019,
            27,  1499, 15780,  1460,  

In [ ]:
import torch

with torch.no_grad():
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state[:, 0, :]

print(embeddings.shape)
print(embeddings)

torch.Size([1, 768])
tensor([[ 1.9433e-01,  1.2383e-02, -4.8061e-01,  3.6040e-01,  2.6636e-01,
          9.9355e-01,  5.1738e-02,  1.9694e-01,  2.1317e-01,  3.6828e-01,
          4.9840e-01,  1.7027e+00, -1.2481e+00, -8.2174e-01, -2.8511e-01,
          4.0814e-01,  1.2675e+00, -3.4413e-01, -4.5543e-01,  4.5066e-01,
          1.1011e+00,  4.1722e-01,  8.2018e-01,  1.7536e-01,  1.8640e+00,
          3.3910e-01,  9.1263e-01,  1.3739e+00, -1.2632e+00, -2.0076e-01,
         -1.1071e+00, -1.3650e+00, -1.5034e-01,  1.4657e+00, -1.7804e+00,
         -1.3426e-01, -3.7453e-01,  4.0156e-01, -1.0548e-01, -1.1302e+00,
          1.2570e+00, -1.7545e+00,  8.4068e-01,  3.5778e-01,  1.5629e-01,
         -6.8012e-01, -6.5777e-01,  3.5761e-01,  2.7908e-01, -1.2653e+00,
         -3.5856e-01,  3.0196e-02, -7.5689e-02, -7.5201e-01,  1.2661e+00,
         -1.0244e+00,  1.4024e-02, -4.1742e-01,  4.2164e-01, -5.7847e-01,
          4.0246e-01,  2.4194e+00, -2.4547e+00,  3.7253e-02,  4.3004e-01,
         -6.9192e